In [4]:
import os
from dotenv import load_dotenv
from aocd import get_data

In [5]:
load_dotenv()
secret_key = os.getenv("SECRET_KEY")
os.environ['AOC_SESSION'] = secret_key

data = get_data(year=2023, day=5)

# Part 1
The almanac (your puzzle input) lists all of the seeds that need to be planted. It also lists what type of soil to use with each kind of seed, what type of fertilizer to use with each kind of soil, what type of water to use with each kind of fertilizer, and so on. Every type of seed, soil, fertilizer and so on is identified with a number, but numbers are reused by each category - that is, soil 123 and fertilizer 123 aren't necessarily related to each other.

For example:
```
seeds: 79 14 55 13

seed-to-soil map:
50 98 2
52 50 48

soil-to-fertilizer map:
0 15 37
37 52 2
39 0 15

fertilizer-to-water map:
49 53 8
0 11 42
42 0 7
57 7 4

water-to-light map:
88 18 7
18 25 70

light-to-temperature map:
45 77 23
81 45 19
68 64 13

temperature-to-humidity map:
0 69 1
1 0 69

humidity-to-location map:
60 56 37
56 93 4
```
The almanac starts by listing which seeds need to be planted: seeds 79, 14, 55, and 13.

The rest of the almanac contains a list of maps which describe how to convert numbers from a source category into numbers in a destination category. That is, the section that starts with seed-to-soil map: describes how to convert a seed number (the source) to a soil number (the destination). This lets the gardener and his team know which soil to use with which seeds, which water to use with which fertilizer, and so on.

Rather than list every source number and its corresponding destination number one by one, the maps describe entire ranges of numbers that can be converted. Each line within a map contains three numbers: the destination range start, the source range start, and the range length.

Consider again the example seed-to-soil map:

50 98 2 52 50 48 The first line has a destination range start of 50, a source range start of 98, and a range length of 2. This line means that the source range starts at 98 and contains two values: 98 and 99. The destination range is the same length, but it starts at 50, so its two values are 50 and 51. With this information, you know that seed number 98 corresponds to soil number 50 and that seed number 99 corresponds to soil number 51.

The second line means that the source range starts at 50 and contains 48 values: 50, 51, ..., 96, 97. This corresponds to a destination range starting at 52 and also containing 48 values: 52, 53, ..., 98, 99. So, seed number 53 corresponds to soil number 55.

Any source numbers that aren't mapped correspond to the same destination number. So, seed number 10 corresponds to soil number 10.

So, the entire list of seed numbers and their corresponding soil numbers looks like this:

seed soil 0 0 1 1 ... ... 48 48 49 49 50 52 51 53 ... ... 96 98 97 99 98 50 99 51 With this map, you can look up the soil number required for each initial seed number:

Seed number 79 corresponds to soil number 81. Seed number 14 corresponds to soil number 14. Seed number 55 corresponds to soil number 57. Seed number 13 corresponds to soil number 13. The gardener and his team want to get started as soon as possible, so they'd like to know the closest location that needs a seed. Using these maps, find the lowest location number that corresponds to any of the initial seeds. To do this, you'll need to convert each seed number through other categories until you can find its corresponding location number. In this example, the corresponding types are:

Seed 79, soil 81, fertilizer 81, water 81, light 74, temperature 78, humidity 78, location 82. Seed 14, soil 14, fertilizer 53, water 49, light 42, temperature 42, humidity 43, location 43. Seed 55, soil 57, fertilizer 57, water 53, light 46, temperature 82, humidity 82, location 86. Seed 13, soil 13, fertilizer 52, water 41, light 34, temperature 34, humidity 35, location 35. So, the lowest location number in this example is 35.

What is the lowest location number that corresponds to any of the initial seed numbers?

In [4]:
def get_seeds_and_maps(data):
    # Parse input to set up seed and maps
    seed_data = data.splitlines()[0]
    map_data = data.splitlines()[1:]

    seeds = [int(x) for x in seed_data.split(': ')[1].split()]

    maps = {}
    current_map = None

    for line in map_data:
        if line[-4:] == 'map:':
            # Get map type
            map_name = line.split(' map:')[0].replace('-', '_')
            # Create dictionary
            maps[map_name] = []
            # Update current_map
            current_map = map_name
        elif len(line.strip()) > 0: # if line isn't blank
            # Parse each map line
            map_info = [int(x) for x in line.split(' ') if x != '']
            destination_start, source_start, range_length = map_info[0], map_info[1], map_info[2]
            maps[current_map].append((source_start, destination_start, range_length))
    return seeds, maps

def get_location(seed, maps):
    source_val = seed
    for map in maps:
        destination_found = False
        for map_info in maps[map]:
            source_start, destination_start, range_length = map_info
            if source_start <= source_val < source_start + range_length:
                destination_val = destination_start + (source_val - source_start)
                destination_found = True
        # If val isn't found in map_info, set destination=source
        if not destination_found: destination_val = source_val
        source_val = destination_val
    return destination_val

def get_min_location(data):
    seeds, maps = get_seeds_and_maps(data)
    locations = []
    for seed in seeds:
        location = get_location(seed, maps)
        locations.append(location)
    min_location = min(locations)
    return min_location

In [6]:
answer = get_min_location(data)
print(answer)

174137457


# Part 2
Everyone will starve if you only plant such a small number of seeds. Re-reading the almanac, it looks like the seeds: line actually describes ranges of seed numbers.

The values on the initial seeds: line come in pairs. Within each pair, the first value is the start of the range and the second value is the length of the range. So, in the first line of the example above:
```
seeds: 79 14 55 13
```
This line describes two ranges of seed numbers to be planted in the garden. The first range starts with seed number 79 and contains 14 values: 79, 80, ..., 91, 92. The second range starts with seed number 55 and contains 13 values: 55, 56, ..., 66, 67.

Now, rather than considering four seed numbers, you need to consider a total of 27 seed numbers.

In the above example, the lowest location number can be obtained from seed number 82, which corresponds to soil 84, fertilizer 84, water 84, light 77, temperature 45, humidity 46, and location 46. So, the lowest location number is 46.

Consider all of the initial seed numbers listed in the ranges on the first line of the almanac. What is the lowest location number that corresponds to any of the initial seed numbers?

In [6]:
sample = '''seeds: 79 14 55 13

seed-to-soil map:
50 98 2
52 50 48

soil-to-fertilizer map:
0 15 37
37 52 2
39 0 15

fertilizer-to-water map:
49 53 8
0 11 42
42 0 7
57 7 4

water-to-light map:
88 18 7
18 25 70

light-to-temperature map:
45 77 23
81 45 19
68 64 13

temperature-to-humidity map:
0 69 1
1 0 69

humidity-to-location map:
60 56 37
56 93 4'''

In [15]:
# Update to handle seed ranges in part 2
def get_seeds_and_maps(data):
    # Parse input to set up seed and maps
    seed_data = data.splitlines()[0]
    map_data = data.splitlines()[1:]

    # Get seed ranges
    seeds = [int(x) for x in seed_data.split(': ')[1].split()]
    seed_pairs = zip(seeds[::2], seeds[1::2])

    maps = {}
    current_map = None

    for line in map_data:
        if line[-4:] == 'map:':
            # Get map type
            map_name = line.split(' map:')[0].replace('-', '_')
            # Create dictionary
            maps[map_name] = []
            # Update current_map
            current_map = map_name
        elif len(line.strip()) > 0: # if line isn't blank
            # Parse each map line
            map_info = [int(x) for x in line.split(' ') if x != '']
            destination_start, source_start, range_length = map_info[0], map_info[1], map_info[2]
            maps[current_map].append((destination_start, source_start, range_length))
    return seed_pairs, maps

def check_overlap(range1: tuple, map_rule: tuple) -> bool:
    """Check if a range overlaps with a map rule's source range."""
    start1, length1 = range1
    dest_start, source_start, range_length = map_rule
    
    # Calculate inclusive end points
    end1 = start1 + length1 - 1
    source_end = source_start + range_length - 1
    
    # Check for overlap
    return max(start1, source_start) <= min(end1, source_end)

def process_range_overlap(seed_range: tuple, map_rule: tuple) -> list:
    """Process overlap between a seed range and a map rule, returning list of new ranges."""
    seed_start, seed_length = seed_range
    dest_start, source_start, rule_length = map_rule
    
    seed_end = seed_start + seed_length - 1
    source_end = source_start + rule_length - 1
    
    ranges = []
    
    # Before overlap (unmapped portion)
    if seed_start < source_start:
        before_length = source_start - seed_start
        ranges.append((seed_start, before_length))
    
    # Overlapping part (mapped portion)
    overlap_start = max(seed_start, source_start)
    overlap_end = min(seed_end, source_end)
    if overlap_start <= overlap_end:
        offset = overlap_start - source_start
        mapped_start = dest_start + offset
        mapped_length = overlap_end - overlap_start + 1
        ranges.append((mapped_start, mapped_length))
    
    # After overlap (unmapped portion)
    if seed_end > source_end:
        after_start = source_end + 1 - source_start + seed_start  # Convert back to seed space
        after_length = seed_end - source_end
        ranges.append((after_start, after_length))
    
    return ranges

def merge_ranges(ranges: list) -> list:
    """Merge overlapping or adjacent ranges and sort them."""
    if not ranges:
        return []
    
    # Sort ranges by start position
    ranges = sorted(ranges, key=lambda x: x[0])
    
    merged = []
    current_start, current_length = ranges[0]
    current_end = current_start + current_length - 1
    
    for start, length in ranges[1:]:
        end = start + length - 1
        if start <= current_end + 1:
            # Ranges overlap or are adjacent, extend current range
            current_end = max(current_end, end)
        else:
            # No overlap, add current range and start new one
            merged.append((current_start, current_end - current_start + 1))
            current_start, current_end = start, end
    
    # Add the last range
    merged.append((current_start, current_end - current_start + 1))
    
    return merged

def get_min_location(seed_pairs: list, maps: dict) -> int:
    """Find the minimum location number for all seed ranges."""
    min_location = float('inf')
    
    for seed_range in seed_pairs:
        current_ranges = [seed_range]
        
        # Process each map type in sequence
        for map_type in maps:
            new_ranges = []
            unmapped_ranges = current_ranges[:]
            
            # Process each map rule
            for map_rule in maps[map_type]:
                temp_unmapped = []
                for range_ in unmapped_ranges:
                    if check_overlap(range_, map_rule):
                        # Get both mapped and unmapped portions
                        processed_ranges = process_range_overlap(range_, map_rule)
                        for processed_range in processed_ranges:
                            if processed_range[0] >= map_rule[1] and \
                               processed_range[0] < map_rule[1] + map_rule[2]:
                                new_ranges.append(processed_range)
                            else:
                                temp_unmapped.append(processed_range)
                    else:
                        temp_unmapped.append(range_)
                unmapped_ranges = temp_unmapped
            
            # Add any remaining unmapped ranges
            new_ranges.extend(unmapped_ranges)
            
            # Merge overlapping ranges before moving to next map
            current_ranges = merge_ranges(new_ranges)
        
        # Update minimum location based on the starts of final ranges
        min_location = min(min_location, min(start for start, _ in current_ranges))
    
    return min_location

def solve_part2(input_data: str) -> int:
    """Solve part 2 of the puzzle."""
    seed_pairs, maps = get_seeds_and_maps(input_data)
    return get_min_location(seed_pairs, maps)

In [16]:
solve_part2(data)

19447031

In [11]:
seed_pairs, maps = get_seeds_and_maps(sample)
answer = get_min_location(seed_pairs, maps)
print(answer)

Processing seed pairs: [(79, 14), (55, 13)]

Starting with range: (79, 14)

Processing map type: seed_to_soil
After mapping: [(81, 14)]

Processing map type: soil_to_fertilizer
After mapping: [(81, 14)]

Processing map type: fertilizer_to_water
After mapping: [(81, 14)]

Processing map type: water_to_light
After mapping: [(74, 14)]

Processing map type: light_to_temperature
After mapping: [(74, 3), (45, 11)]

Processing map type: temperature_to_humidity
After mapping: [(74, 3), (46, 11)]

Processing map type: humidity_to_location
After mapping: [(78, 3), (46, 10), (60, 1)]

Starting with range: (55, 13)

Processing map type: seed_to_soil
After mapping: [(57, 13)]

Processing map type: soil_to_fertilizer
After mapping: [(57, 13)]

Processing map type: fertilizer_to_water
After mapping: [(53, 4), (61, 9)]

Processing map type: water_to_light
After mapping: [(46, 4), (54, 9)]

Processing map type: light_to_temperature
After mapping: [(82, 4), (90, 9)]

Processing map type: temperature_to_

In [17]:
def get_seeds_and_maps(data):
    # Parse input to set up seed and maps
    seed_data = data.splitlines()[0]
    map_data = data.splitlines()[1:]
    # Get seed ranges
    seeds = [int(x) for x in seed_data.split(': ')[1].split()]
    seed_pairs = list(zip(seeds[::2], seeds[1::2]))
    
    maps = {}
    current_map = None
    for line in map_data:
        if line.strip() == '':
            continue
        if line[-4:] == 'map:':
            # Get map type
            map_name = line.split(' map:')[0].replace('-', '_')
            # Create dictionary
            maps[map_name] = []
            # Update current_map
            current_map = map_name
        elif len(line.strip()) > 0:  # if line isn't blank
            # Parse each map line
            map_info = [int(x) for x in line.split()]
            destination_start, source_start, range_length = map_info
            maps[current_map].append((destination_start, source_start, range_length))
    
    # Sort map rules by source_start for efficient processing
    for map_type in maps:
        maps[map_type].sort(key=lambda x: x[1])
    
    return seed_pairs, maps

def check_overlap(range1: tuple, map_rule: tuple) -> bool:
    """Check if a range overlaps with a map rule's source range."""
    start1, length1 = range1
    dest_start, source_start, range_length = map_rule
    
    # Calculate inclusive end points
    end1 = start1 + length1 - 1
    source_end = source_start + range_length - 1
    
    # Check for overlap
    return max(start1, source_start) <= min(end1, source_end)

def process_range_overlap(seed_range: tuple, map_rule: tuple) -> list:
    """Process overlap between a seed range and a map rule, returning list of new ranges."""
    seed_start, seed_length = seed_range
    dest_start, source_start, rule_length = map_rule
    
    seed_end = seed_start + seed_length - 1
    source_end = source_start + rule_length - 1
    
    ranges = []
    
    # Before overlap (unmapped portion)
    if seed_start < source_start:
        before_length = source_start - seed_start
        ranges.append((seed_start, before_length))
    
    # Overlapping part (mapped portion)
    overlap_start = max(seed_start, source_start)
    overlap_end = min(seed_end, source_end)
    if overlap_start <= overlap_end:
        offset = overlap_start - source_start
        mapped_start = dest_start + offset
        mapped_length = overlap_end - overlap_start + 1
        ranges.append((mapped_start, mapped_length))
    
    # After overlap (unmapped portion)
    if seed_end > source_end:
        after_start = source_end + 1 - source_start + seed_start  # Convert back to seed space
        after_length = seed_end - source_end
        ranges.append((after_start, after_length))
    
    return ranges

def merge_ranges(ranges: list) -> list:
    """Merge overlapping or adjacent ranges and sort them."""
    if not ranges:
        return []
    
    # Sort ranges by start position
    ranges = sorted(ranges, key=lambda x: x[0])
    
    merged = []
    current_start, current_length = ranges[0]
    current_end = current_start + current_length - 1
    
    for start, length in ranges[1:]:
        end = start + length - 1
        if start <= current_end + 1:
            # Ranges overlap or are adjacent, extend current range
            current_end = max(current_end, end)
        else:
            # No overlap, add current range and start new one
            merged.append((current_start, current_end - current_start + 1))
            current_start, current_end = start, end
    
    # Add the last range
    merged.append((current_start, current_end - current_start + 1))
    
    return merged

def get_min_location(seed_pairs: list, maps: dict) -> int:
    """Find the minimum location number for all seed ranges."""
    min_location = float('inf')
    
    for seed_range in seed_pairs:
        current_ranges = [seed_range]
        
        # Process each map type in sequence
        for map_type in maps:
            new_ranges = []
            unmapped_ranges = current_ranges[:]
            
            # Process each map rule
            for map_rule in maps[map_type]:
                temp_unmapped = []
                for range_ in unmapped_ranges:
                    if check_overlap(range_, map_rule):
                        # Get both mapped and unmapped portions
                        processed_ranges = process_range_overlap(range_, map_rule)
                        for processed_range in processed_ranges:
                            if processed_range[0] >= map_rule[1] and \
                               processed_range[0] < map_rule[1] + map_rule[2]:
                                new_ranges.append(processed_range)
                            else:
                                temp_unmapped.append(processed_range)
                    else:
                        temp_unmapped.append(range_)
                unmapped_ranges = temp_unmapped
            
            # Add any remaining unmapped ranges
            new_ranges.extend(unmapped_ranges)
            
            # Merge overlapping ranges before moving to next map
            current_ranges = merge_ranges(new_ranges)
        
        # Update minimum location based on the starts of final ranges
        min_location = min(min_location, min(start for start, _ in current_ranges))
    
    return min_location

def solve_part2(input_data: str) -> int:
    """Solve part 2 of the puzzle."""
    seed_pairs, maps = get_seeds_and_maps(input_data)
    return get_min_location(seed_pairs, maps)

solve_part2(data)

15252996